# Imports

In [ ]:
%matplotlib inline

import json
import copy
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tifffile
from skimage.segmentation import find_boundaries
from skimage.morphology import dilation
import tqdm.auto as tqdm

# Define Heatmap Generation Functions

In [ ]:
def make_heatmap_fig(anno_file,
                     cell_counts,
                     zs,
                     fig_layout=None,
                     fig_size=[8,8],
                     cbar_dims=[0.725, 0.075, 0.15, 0.025],
                     cmap="hot_r",
                     col_name="density",
                     col_multiplier=1,
                     legend_title="Cells per $mm^3$",
                     legend_fontsize=20,
                     legend_numsize=18,
                     round_to=10,
                     include_boxes=False,
                     output_file=""):
    """Generate a standarized heatmap figure with potentially multiple z-planes
    Parameters
    ----------
    anno_file : str
        A file path to an annotation TIFF to use as a baseline regions map
    cell_counts : str
        A file path to a regions-specific metric CSV file (as generated by EFLASH or SHIELD)
    zs : List[int]
        A list of z-values for which heatmaps should be generated
    fig_layout : Union[List[int], Tuple[int, int]]
        A pair of integers specifying the number of heatmaps tall and wide to make the figure (in that order)
    fig_size : Union[List[Union[int, float]], Tuple[Union[int, float]], Union[int, float]]]
        A pair of integers or floats specifying the width and height of the figure (in inches)
    cbar_dims : Union[List[float], Tuple[float, float, float, float]]
        Specifies the position and dimensions of the color legend, as proportions of figure size
        Format: [left offset, bottom offset, width, height]
    cmap : str
        A matplotlib-recognizable colormap name
    col_name : str
        The name of the column of the region-specific CSV file to use as a metric
    col_multiplier : Union[int, float]
        Optional multiplicative correction to numeric column data
    legend_title : str
        Text to display above the legend, LaTeX-style formatting allowed
    legend_fontsize : Union[int, float]
        Font size of the legend title
    legend_numsize : Union[int, float]
        Font size of the legend tick number
    round_to : Union[int, float]
        The upper bound of the legend will be rounded to the nearest multiple of this number
    include_boxes : bool
        If True, subplot axes will be displayed. 
        Set to True for rapid prototyping of figure dimensions and layouts
    output_file : str
        File path to an image file to which the figure should be saved
        Use an enpty string to specify no file saving
    
    Returns
    -------
    None
    """
    if fig_layout is None:
        fig_layout = (len(zs), 1)
    
    if fig_layout[0] * fig_layout[1] < len(zs):
        raise ValueError("{} plot layout doens't have enough subplots for {} slices!".format(fig_layout, len(zs)))  
    
    anno = tifffile.imread(anno_file).astype(np.int)
    
    # Reads in cell counts file
    # Fills in empty regions with 0
    # Rewrites as dictionary to use as key-value mapping
    counts = pd.read_csv(cell_counts)
    counts[col_name] *= col_multiplier
    id_max = np.max(anno)
    new_index = pd.Index(range(id_max + 1), name="id")
    counts_list = (counts[["id", col_name]].set_index("id")          
                                           .reindex(new_index)      # Fills in missing region IDs
                                           .reset_index()           # Sets missing regions to empty
                                           .fillna(0))              # Sets empty regions to 0
    counts_list = counts_list.values.tolist()
    
    # Uses dictionary key:val pairs to fill in z-plane with proper density values
    counts_dict = {int(old):new for old, new in counts_list}
    
    # Sets root's density value to 0
    counts_dict[0] = 0
    
    # Generates custom color map for boundaries
    # Sets non-boundary regions to be transparent
    custom_cmap = copy.copy(plt.cm.get_cmap("Greys"))
    custom_cmap.set_bad(alpha=0)
    
    # Calculates upper bound for legend and heatmap color map
    density_bound = round_to * (round(max(counts[col_name]) * 1.1 / round_to) + 1)
    
    # If only one z-plane, does not enumerate through subplot axes
    # Required because plt.subplots with one subplot does not return iterable axes object
    if len(zs) == 1:
        fig, ax = plt.subplots(*fig_layout, figsize=fig_size)
        plt.axis("off")
    
        # Uses annotation to find boundaries of given z-slice
        # Sets non-boundary regions to NaN
        z = zs[0]
        anno_slice = anno[z]
        boundaries = find_boundaries(anno_slice, mode="thick").astype(np.float)
        boundaries = dilation(boundaries)
        boundaries[boundaries < 0.5] = np.nan
        
        # Vectorized substitution of density values for annotation regions
        # Modified from: https://stackoverflow.com/q/55949809/
        k = np.array(list(counts_dict.keys()))
        v = np.array(list(counts_dict.values()))
        mapping_ar = np.zeros(k.max() + 1, dtype=v.dtype)
        mapping_ar[k] = v
        new_slice = mapping_ar[anno_slice].astype(np.float)
        
        # Add heatmap and boundaries to figure
        im = ax.imshow(new_slice, cmap=cmap, vmin=0, vmax=density_bound)
        ax.imshow(boundaries, cmap=custom_cmap, vmin=0, vmax=1, alpha=0.4)   
        
        if not include_boxes:
            ax.axis("off")
        
        # Add colorbar to figure
        cbaxes = fig.add_axes(cbar_dims)
        cb = plt.colorbar(im, orientation="horizontal", ticks=[0, round(density_bound)], cax=cbaxes)
        cb.outline.set_visible(False)
        cb.ax.tick_params(size=0, labelsize=18)
        cb.ax.set_title(legend_title, fontsize=20)
                
        
    else:
        fig, axes = plt.subplots(*fig_layout, figsize=fig_size)
        plt.axis("off")
        
        # Enumerates through z-planes and subplot axes
        # See len(zs) == 1 case above for documentation
        for i, (z, ax) in enumerate(zip(zs, axes.reshape(-1))):
            anno_slice = anno[z]
            boundaries = find_boundaries(anno_slice, mode="thick").astype(np.float)
            boundaries = dilation(boundaries)
            boundaries[boundaries < 0.5] = np.nan

            k = np.array(list(counts_dict.keys()))
            v = np.array(list(counts_dict.values()))
            mapping_ar = np.zeros(k.max() + 1, dtype=v.dtype)
            mapping_ar[k] = v
            new_slice = mapping_ar[anno_slice].astype(np.float)

            im = ax.imshow(new_slice, cmap=cmap, vmin=0, vmax=density_bound)
            ax.imshow(boundaries, cmap=custom_cmap, vmin=0, vmax=1, alpha=0.5)

            if not include_boxes:
                ax.axis("off")

            plt.subplots_adjust(wspace=0, hspace=0)

        cbaxes = fig.add_axes(cbar_dims)
        cb = plt.colorbar(im, orientation="horizontal", ticks=[0, round(density_bound)], cax=cbaxes)
        cb.outline.set_visible(False)
        cb.ax.tick_params(size=0, labelsize=legend_numsize)
        cb.ax.set_title(legend_title, fontsize=legend_fontsize)
    
    # Save figure if specified
    if output_file:
        plt.savefig(output_file, dpi=300)

    plt.show()
        

In [ ]:
def write_one_heatmap(anno, 
                      counts,
                      counts_dict, 
                      z, 
                      border_cmap, 
                      filename, 
                      dpi=300, 
                      fig_size=[8,8], 
                      cbar_dims=[0.9, 0.15, 0.05, 0.6],
                      cmap="hot_r",
                      col_name="density",
                      legend_title="Cells per $mm^3$",
                      round_to=10, 
                      flip_x=False, 
                      flip_y=False):
    """Writes out a standardized heatmap image for a single z-plane
    
    Parameters
    ----------
    anno : np.ndarray
        3D annotation array in numpy ndarray format, with axes in [x, y, x] order
    counts : pd.DataFrame
        Region-specific metric CSV in pandas DataFrame format
    counts_dict : dict
        Dictionary where keys are region IDs, values are metric values
    z : int
        z-depth of heatmap    
    border_cmap : matplotlib.colors.LinearSegmentedColormap
        Colormap for border, NaN's should be set to transparent
    filename : str
        File name/path to which to write figure    
    dpi : Union[int, float]
        DPI of saved figure    
    fig_size : Union[List[Union[int, float]], Tuple[Union[int, float]], Union[int, float]]]
        A pair of integers or floats specifying the width and height of the figure (in inches)
    cbar_dims : Union[List[float], Tuple[float, float, float, float]]
        Specifies the position and dimensions of the color legend, as proportions of figure size
        Format: [left offset, bottom offset, width, height]  
    cmap : str
        A matplotlib-recognizable colormap name
    col_name : str
        The name of the column of the region-specific CSV file to use as a metric
    legend_title : str
        Text to display above the legend, LaTeX-style formatting allowed
    round_to : Union[int, float]
        The upper bound of the legend will be rounded to the nearest multiple of this number
    flip_x : bool
        True if the x-dimension should be reversed    
    flip_y : bool
        True if the y-dimension should be reversed  
       
    Returns
    -------
    None   
    """
    density_bound = round_to * (round(max(counts[col_name]) * 1.1 / round_to) + 1)
    
    fig, ax = plt.subplots(figsize=fig_size)
    plt.axis("off")

    anno_slice = anno[z]
    boundaries = find_boundaries(anno_slice, mode="thick").astype(np.float)
    boundaries = dilation(boundaries)
    boundaries[boundaries < 0.5] = np.nan

    k = np.array(list(counts_dict.keys()))
    v = np.array(list(counts_dict.values()))
    mapping_ar = np.zeros(k.max() + 1, dtype=v.dtype)
    mapping_ar[k] = v
    new_slice = mapping_ar[anno_slice].astype(np.float)
    
    if flip_x:
        new_slice = np.flip(new_slice, 1)
        boundaries = np.flip(boundaries, 1)
    if flip_y:
        new_slice = np.flip(new_slice, 0)
        boundaries = np.flip(boundaries, 0)

    im = plt.imshow(new_slice, cmap=cmap, vmin=0, vmax=density_bound)
    plt.imshow(boundaries, cmap=custom_cmap, vmin=0, vmax=1, alpha=0.4)   

    cbaxes = fig.add_axes(cbar_dims)
    cb = plt.colorbar(im, ticks=[0, round(density_bound)], cax=cbaxes)
    cb.outline.set_visible(False)
    cb.ax.tick_params(size=0, labelsize=20)
    cb.ax.set_title(legend_title, fontsize=24, pad=18)
    
    plt.savefig(filename, dpi=dpi, bbox_inches="tight")
    plt.close()

# Generate Cell Count Heatmaps

In [ ]:
# Input Files
# The annotation file should correspond to the view desired in the figure,
# it does not necessarily need to correspond with the view of the original data. 
anno_file = "../atlas/annotation_10_full_coronal_sym.tiff"
cell_counts = "cell_counts_level-10_sym.csv"

# List of Zs to use in generating heatmap file
zs = np.arange(200, 1301, 150)
print("This z range will generate {} images.".format(len(zs)))
print("Z range: {}".format(zs))

# Layout of figure (format: [n height, n width], where (n height)*(n width) = length of z range
fig_layout = [2, 4]

# Figure Size (format: [width (inches), height (inches)])
fig_size = [20, 7.25]

# Include Boxes (draws boxes around each heatmap, useful for adjusting figure dimensions and spacing)
include_boxes = False

# Color map (using matplotlib color maps)
cmap = "hot_r"

# Column Name ("density" for cell counts, "mean_intensity" for intensities)
col_name = "density"

# Column multiplier (corrects for density adjustments during cell/intentensity counting)
# Adjusts such that units of legend are in units/mm^3
# Use 2.5*2.5*2.5 for 25 um annotation atlas alignment/counting
col_multiplier = 2.5*2.5*2.5

# Legend title
# Recommend using "Cells per $mm^3$" for cells, "Intensity per $mm^3$" for intensity
legend_title = "Cells per $mm^3$"

# Legend title font size
legend_fontsize = 20

# Legend tick numbers font size
legend_numsize = 18

# Round legend upper bound to nearest....
round_to = 500

# Legend dimensions (Format: [left offset, bottom offset, width, height])
# Where width and height are proportions of figure width and height
cbar_dims = [0.725, 0.035, 0.15, 0.025]

# Output file path (leave as empty string if no output file desired)
output_file = ""

In [ ]:
make_heatmap_fig(anno_file=anno_file,
                 cell_counts=cell_counts,
                 zs=zs,
                 fig_layout=fig_layout,
                 fig_size=fig_size,
                 col_name=col_name,
                 col_multiplier=col_multiplier,
                 legend_title=legend_title,
                 legend_fontsize=legend_fontsize,
                 legend_numsize=legend_numsize,
                 round_to=round_to,
                 include_boxes=include_boxes,
                 cbar_dims=cbar_dims)

# Generate Image Series for Heatmap Animation

In [ ]:
# Input files
anno_file = "../atlas/annotation_10_full_coronal_LR.tiff"
cell_counts = "cell_counts_level-10_LR.csv"

# Reverse order of z slices
flip_z = False

# Select z range
zs = list(range(0, 1317, 3))
if flip_z:
    zs.reverse()
print("This z range will generate {} images.".format(len(zs)))
print("Z range: [{}, {}, {}, ..., {}, {}, {}]".format(zs[0], zs[1], zs[2], zs[-3], zs[-2], zs[-1]))

# Figure Size (format: [width (inches), height (inches)])
fig_size = [8, 8]

# Color map (using matplotlib color maps)
cmap = "hot_r"

# Column Name ("density" for cell counts, "mean_intensity" for intensities)
col_name = "density"

# Column multiplier (corrects for density adjustments during cell/intentensity counting)
# Adjusts such that units of legend are in units/mm^3
# Use 2.5*2.5*2.5 for 25 um annotation atlas alignment/counting
col_multiplier = 2.5*2.5*2.5

# Legend title
# Recommend using "Cells per $mm^3$" for cells, "Intensity per $mm^3$" for intensity
legend_title = "Cells per $mm^3$"

# Legend dimensions (Format: [left offset, bottom offset, width, height])
# Where width and height are proportions of figure width and height
cbar_dims = [0.9, 0.15, 0.05, 0.6]

# Round legend upper bound to nearest....
round_to = 500

# Flip about x-axis
flip_x = False

# Flip about y-axis
flip_y = False

# Output directory
output_dir = "heatmap_anim_pngs"

# DPI of output images
dpi = 300

# Output file extension (we recommend ".png") 
output_ext = ".png"

In [ ]:
# Rerun this cell whenever the annotation file or input file (cell count or intensity) changes

# Load annotation and heatmap legend file (cell counts or intensities)
anno = tifffile.imread(anno_file).astype(np.int)
counts = pd.read_csv(cell_counts)

# Process Annotation File and Generate Mapping Index
counts[col_name] *= col_multiplier
id_max = np.max(anno)
new_index = pd.Index(range(id_max + 1), name="id")
counts_list = (counts[["id", col_name]].set_index("id")
                                        .reindex(new_index)
                                        .reset_index()
                                        .fillna(0))
counts_list = counts_list.values.tolist()
counts_dict = {int(old):new for old, new in counts_list}
counts_dict[0] = 0

# Generate Custom Heatmap for Boundaries
custom_cmap = copy.copy(plt.cm.get_cmap("Greys"))
custom_cmap.set_bad(alpha=0)

In [ ]:
# Generate Heatmap Images

# Create output directory if it does not exist
if not os.path.exists(output_dir):
    print("Creating directory at {}.".format(output_dir))
    os.mkdir(output_dir)

for i, z in enumerate(tqdm.tqdm(zs)):
    filename = output_dir + "/img_" + str(i).zfill(4) + output_ext
    write_one_heatmap(anno=anno, 
                      counts=counts,
                      counts_dict=counts_dict, 
                      z=z, 
                      border_cmap=custom_cmap, 
                      filename=filename, 
                      dpi=300, 
                      fig_size=fig_size,
                      cbar_dims=cbar_dims,
                      cmap=cmap,
                      round_to=round_to,
                      flip_x=False,
                      flip_y=False)

The following terminal command can be used to stitch the PNGs together into an MP4 file (on Windows and MacOS, you may need to install `ffmpeg`):

`ffmpeg -r [frame rate] -pattern_type glob -i "[path-to-data]/img_*.[file-ext]" -vf "pad=width=[x-pixels]:height=[y-pixels]:x=[x-offset]:y=[y-offset]:color=white" -pix_fmt yuv420p [filename.mp4]`

Where:
* `[frame rate]` is the number of PNGs that make up each second of video. 

* `[path-to-data]/img_*.[file-ext]` is a UNIX-style glob expression to the collection of image files.

* `[x-pixels]` is an *even* number slighly larger than the number of pixels in the x-dimension. Similarly, `[y-pixels]` is an *even* number slightly larger than the number of pixels in the y-dimension. `[x-offset]` and `[y-offset]` are used to center the original image. 

For example, if the generated image files are `3121 x 1925`, we might use `x-pixels = 3124`, `y-pixels = 1926`, `x-offset = 1`, and `y-offset = 1`.

This entire process of specifying *even* dimensions and offsets is because the rendering algorithm for turning PNGs into an MP4 file requires an even number of pixels in each dimension. 